In [1]:
import torch
from torchvision import transforms
import numpy as np
from tqdm import tqdm
from PIL import Image
from RNN.RNN4 import *


In [2]:
def benchmark_it_RNN(data_loader, model, device):
    differences = []
    model.eval() 
    
    with torch.no_grad():
        hidden_state = None 

        for images, labels, actions in tqdm(data_loader):
            images, labels, actions = images.to(device), labels.to(device), actions.to(device)

            predictions, hidden_state = model(images, actions, hidden_state)

            hidden_state = tuple(h.detach() for h in hidden_state)

            predictions = predictions.squeeze(-1)

            for pred, true_label in zip(predictions.view(-1), labels.view(-1)):
                difference = abs((pred - true_label).item())
                differences.append(difference)

    benchmark_score = np.mean(np.abs(differences))

    return benchmark_score, differences

In [3]:
model_path = "lane_detection_rnn_new_RNN4.pth"
input_shape = (3, 480, 640) 
rnn_hidden_size = 128

device = "cpu" #torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = load_RNN_model("RNN/models/"+model_path, input_shape, device, rnn_hidden_size)

images_dir = "RNN/training_images/benchmark/images"
labels_dir = "RNN/training_images/benchmark/labels"
actions_dir = "RNN/training_images/benchmark/actions"
transform = get_transform()

seq_length = 20 
batch_size = 1 

benchmark_loader = get_sequential_dataloader(
    images_dir, labels_dir,actions_dir, batch_size=batch_size, seq_length=seq_length,
    train_fraction=0.0, val_fraction=0.0, test_fraction=1.0 
)[2]


RNN model loaded successfully from 'RNN/models/lane_detection_rnn_new_RNN4.pth'


/home/navid/Desktop/PID/PID (Copy)/PID_final_project/learning/RNN/RNN4.py:587: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map

In [4]:
benchmark_score, differences = benchmark_it_RNN(benchmark_loader, model, device)

print(f"benchmark score (average difference): {benchmark_score:.8f}")


100%|██████████| 50/50 [01:20<00:00,  1.60s/it]

benchmark score (average difference): 0.01771347
